In [5]:
import advent

In [17]:
def get(op, code, pointer, offset):
    # e.g. get('1002', code, 2) -> code[p] (immediate mode)
    # e.g. get('2', code, 2) -> code[code[p]] (parameter mode)
    mode = op % (10**(2+offset)) // (10**(1+offset))
    #print(f"{op}, {pointer}, {offset}, {mode}, {code[pointer+offset]}")
    if mode == 0: # parameter mode
        return code[code[pointer+offset]]
    elif mode == 1: # immediate mode
        return code[pointer+offset]
    raise ValueError(f"{op}, {pointer}, {offset} not allowed")

class IO():
    # Simple implementation of IO: just has static input in the __init__

    def __init__(self, input_):
        self.input_buffer = input_.copy()
        self.output_buffer = []
    
    def add_input(self, value):
        self.input_buffer.append(value)
        return self

    def read(self):
        return self.input_buffer.pop(0)
    
    def write(self, value):
        self.output_buffer.append(value)
        return self
    
    def output(self):
        return self.output_buffer

def step(code, p, io):
    # INPLACE does a intcode step, and returns new state and new p
    # p = -1 means the program is done
    op = code[p]

    #print(op, code)
    if op % 100 == 1: # add
        code[code[p+3]] = get(op, code, p, 1) + get(op, code, p, 2)
        return code, p+4, io
    elif op % 100 == 2: # mul
        code[code[p+3]] = get(op, code, p, 1) * get(op, code, p, 2)
        return code, p+4, io
    elif op % 100 == 3: # read
        code[code[p+1]] = io.read()
        return code, p+2, io
    elif op % 100 == 4: # write
        return code, p+2, io.write(get(op, code, p, 1))
    elif op % 100 == 5: # jmp_f
        if get(op, code, p, 1) != 0:
            return code, get(op, code, p, 2), io
        return code, p+3, io
    elif op % 100 == 6: # jmp_t
        if get(op, code, p, 1) == 0:
            return code, get(op, code, p, 2), io
        return code, p+3, io
    elif op % 100 == 7: # lt
        code[code[p+3]] = int(get(op, code, p, 1) < get(op, code, p, 2))
        return code, p+4, io
    elif op % 100 == 8: # eq
        code[code[p+3]] = int(get(op, code, p, 1) == get(op, code, p, 2))
        return code, p+4, io
    elif op % 100 == 99: # exit
        return code, -1, io
    raise ValueError(f"Incorrect program. Op is {op}")

def run(code, io):
    pointer = 0
    while pointer >= 0:
        code, pointer, io = step(code, pointer, io)
    return code, io.output()


In [9]:
from itertools import permutations


def run_for7(code, thrusters):
    o = run(code.copy(), IO([thrusters[0], 0]))[1][0]
    o = run(code.copy(), IO([thrusters[1], o]))[1][0]
    o = run(code.copy(), IO([thrusters[2], o]))[1][0]
    o = run(code.copy(), IO([thrusters[3], o]))[1][0]
    o = run(code.copy(), IO([thrusters[4], o]))[1][0]
    return o


data = advent.get_intcode(7)
#data = [3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0]

max_score = 0
max_input = None
for input in permutations([0, 1, 2, 3, 4]):
    score = run_for7(data, input)
    if score > max_score:
        max_score = score
        max_input = list(input)

print(max_score, max_input)

17406 [2, 4, 1, 0, 3]


In [29]:
# We keep running all 5 programs and passing input and output between them
# When program 5 halts, the final output is the final output of program 5
# Because of the one-directional feedback loop, we can luckily run the programs in order:
# First run A, when it has output, start running B, etcetera.

def run_until_output(code, pointer, io):
    io_out_len = len(io.output())
    while pointer >= 0 and len(io.output()) == io_out_len:
        code, pointer, io = step(code, pointer, io)
    return code, pointer, io

def run_for7_2(data, thrusters):
    thruster_a = data.copy(), 0, IO([thrusters[0]])
    thruster_b = data.copy(), 0, IO([thrusters[1]])
    thruster_c = data.copy(), 0, IO([thrusters[2]])
    thruster_d = data.copy(), 0, IO([thrusters[3]])
    thruster_e = data.copy(), 0, IO([thrusters[4]])
    thruster_e[2].write(0) # Pretend as if thruster e outputs 0 to start with

    while thruster_e[1] >= 0:
        thruster_a = run_until_output(thruster_a[0], thruster_a[1], thruster_a[2].add_input(thruster_e[2].output()[-1]))
        thruster_b = run_until_output(thruster_b[0], thruster_b[1], thruster_b[2].add_input(thruster_a[2].output()[-1]))
        thruster_c = run_until_output(thruster_c[0], thruster_c[1], thruster_c[2].add_input(thruster_b[2].output()[-1]))
        thruster_d = run_until_output(thruster_d[0], thruster_d[1], thruster_d[2].add_input(thruster_c[2].output()[-1]))
        thruster_e = run_until_output(thruster_e[0], thruster_e[1], thruster_e[2].add_input(thruster_d[2].output()[-1]))

    return thruster_e[2].output()[-1]

data = advent.get_intcode(7)
#data = [3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5]

max_score = 0
max_input = None
for input in permutations([5, 6, 7, 8, 9]):
    score = run_for7_2(data, input)
    if score > max_score:
        max_score = score
        max_input = list(input)

print(max_score, max_input)


1047153 [7, 8, 6, 9, 5]
